In [ ]:
from init import init
init()
from dl import TorchApproximator
from scipy.stats import norm
import os
import numpy as np
import pandas as pd
from utils import Timer

os.getpid()

In [ ]:
def ds(K, S, T, vol, r, q):
    vol_T = vol * np.sqrt(T)
    d1 = (np.log(S / K) + (r - q + 0.5 * vol * vol) * T) / vol_T
    d2 = d1 - vol_T
    return d1, d2


def put(K, S, T, vol, r, q):
    disc = np.exp(-r * T)
    pv_K = K * disc
    spot_after_div = S * np.exp(-q * T)

    d1, d2 = ds(K, S, T, vol, r, q)
    v = norm.cdf(-d2) * pv_K - norm.cdf(-d1) * spot_after_div
    return v * 100.


v_put = np.vectorize(put)

In [ ]:
%%time
csv_file = 'put_prices_10M.csv'
if os.path.isfile(csv_file):
    df = pd.read_csv(csv_file)
else:
    seed = 314
    np.random.seed(seed)

    n_samples = 10*1000*1000  # Total number of samples
    domain = {
        'spot': (0.5, 2),
        'time': (0, 3.0),
        'sigma': (0.1, 0.5),
        'rate': (-0.01, 0.03),
        'div': (0, 0.02)
    }
    samples = np.zeros(shape=(len(domain.keys()), n_samples))
    for i, r in enumerate(domain.values()):
        samples[i] = np.random.uniform(r[0], r[1], n_samples)
    values = v_put(K=1, S=samples[0], T=samples[1], vol=samples[2], r=samples[3], q=samples[4])
    df = pd.DataFrame.from_dict({'PV' : values,'S':samples[0], 'T':samples[1], 'vol':samples[2], 'r':samples[3], 'q':samples[4]})
    df.to_csv(csv_file, index=False)
    df.head()    

In [ ]:
def run_scenario(scenario):
    with Timer('Train'):
        approximator = TorchApproximator()
        checkpoint, history = approximator.train(df.iloc[:, df.columns != 'PV'].T, df.PV, **scenario)
        model = approximator.load_model(checkpoint)
        original, y_approx = approximator.validation_set(model)
        error = y_approx - original

    mean_err = np.mean(error)
    std_error = np.std(error)
    return (mean_err, std_error), approximator

In [ ]:
default_scenario = {
    'n_epochs': 4000,
    'n_layers': 4,
    'n_hidden': 100
}
results =[]
scenarios = [
    {'n_layers':2, 'n_hidden':100}, 
]
for scenario in scenarios:
    scenario =  {**default_scenario, **scenario }
    result, approximator = run_scenario(scenario)
    print( result, scenario )
    results.append((result, scenario))

data = [{**{'mean':abs(x[0]), 'std':x[1]}, **y} for x,y in results]
df2 = pd.DataFrame(data)
df2